In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import concurrent.futures
import pandas as pd
import geopandas as gpd
from pathlib import Path
import ee
import pickle
from tqdm.auto import tqdm

import scripts.get_sources as get_sources

ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")

/home/dguerrero/module-venv/gwl-modeling/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# make dir to store pickle files
training_path = Path('data/8_training_data')
training_path.mkdir(parents=True, exist_ok=True)

In [21]:
def get_explanatories(row):
    
    date, lon, lat = row['date'], row["lon"], row["lat"]
    aoi = ee.Geometry.Point(lon, lat)
    
    if not get_sources.is_s1_image(date, aoi):
        print("No S1 image for", date)
        return
    
    # TS scans will include the s1 image.

    def get_images(get_image):

        print("Processing", get_image.__name__)
        
        image = get_image(date, aoi)
        
        result = (
             image.reduceRegion(
                reducer=ee.Reducer.toList(),
                geometry=aoi.buffer(50),
                maxPixels=1e13,
                scale=100
            )).getInfo()
        
        print("result", result)
        
        return result
    
    sources = [
        get_sources.get_s1_image,
        get_sources.get_gldas,
        get_sources.get_gpm,
        get_sources.get_srtm,
        get_sources.get_globcover,
        get_sources.get_gedi,
        get_sources.get_hansen
    ]

    with concurrent.futures.ThreadPoolExecutor() as executor:

        futures = {
            executor.submit(get_images, source): source.__name__ for source in sources
        }

        result = {}

        for future in concurrent.futures.as_completed(futures):
            future_name = futures[future]
            result[future_name] = future.result()

    return result

In [22]:
df = pd.read_csv('data/field_data_all.csv')
high_corr_ids = pd.read_csv('data/ids_high_corr.csv')

df = df[df['id'].isin(high_corr_ids['id'])]

# Remove those date where the gwl measure is out of reasonable range
upper_thres = 20
lower_thres = -100

df = df[(df.gwl_cm < upper_thres) & (df.gwl_cm > lower_thres)]

# save the final points

df.to_csv('field_data_high_corr.csv', index=False)

In [23]:
with tqdm(total=len(df)) as pbar:

    with concurrent.futures.ThreadPoolExecutor() as executor:
        
        futures = {}
        for _, row in df.iterrows():
            future_name = f"{row['id']}_{row['date']}"
            future_result_path = training_path / f'{future_name}.p'
            
            if future_result_path.exists():
                pbar.update(1)
                continue

            futures[executor.submit(get_explanatories, row)] = future_name
        
        result = {}

        for future in concurrent.futures.as_completed(futures):

            future_name = futures[future]
            if future.result() is not None:

                # merge all the sub dicts into one dict
                result[future_name] = {k: v for d in list(future.result().values()) for k, v in d.items()}
                
                future_result_path = training_path / f'{future_name}.p'

                # Save the results to a file.
                with open(future_result_path, 'wb') as f:
                    pickle.dump(result, f)
            
            pbar.update(1)

  0%|          | 0/19904 [00:00<?, ?it/s]

No S1 image for 2018-11-05
No S1 image for 2018-11-15
Processing get_s1_image
No S1 image for 2018-11-07
Processing get_gldas
Processing get_gpm
No S1 image for 2018-11-08
No S1 image for 2018-11-09
Processing get_srtm
Processing get_globcover
Processing get_gedi
Processing get_hansen
Processing get_s1_image
Processing get_gldas
Processing get_gpm
Processing get_srtm
No S1 image for 2018-11-06
Processing get_globcover
Processing get_gedi
Processing get_hansen
No S1 image for 2018-11-16


  0%|          | 8/19904 [00:03<2:01:57,  2.72it/s] 

result {'B3': [20], 'B4': [94], 'B5': [79], 'B7': [35], 'ndbri': [0.45736435055732727], 'ndmi': [0.08670520037412643], 'ndvi': [0.6491228342056274]}
result {'prec_3': [0.8589665931649506], 'prec_30': [0.4423375053156633], 'prec_7': [0.6814493386294035], 'precipitation': [1.9706513641609087]}
No S1 image for 2018-11-17


  0%|          | 10/19904 [00:04<1:41:47,  3.26it/s]

...

No S1 image for 2020-02-09
result {'LIA': [], 'VH': [], 'VV': [], 'VVVH_ratio': [], 'angle': [], 'layover': [], 'no_data_mask': [], 'shadow': []}


  1%|          | 234/19904 [01:15<4:44:59,  1.15it/s]

No S1 image for 2020-02-10


  1%|          | 235/19904 [01:16<5:00:05,  1.09it/s]

No S1 image for 2020-02-11
Processing get_s1_image
Processing get_gldas
Processing get_gpm
Processing get_srtm
Processing get_globcover
Processing get_gedi
Processing get_hansen
result {'aspect': [], 'elevation': [], 'slope': []}
result {'B3': [19], 'B4': [77], 'B5': [77], 'B7': [35], 'ndbri': [0.375], 'ndmi': [0], 'ndvi': [0.6041666865348816]}
result {'landcover': [14], 'qa': [0]}
result {'canopy_height': [5]}
result {'prec_3': [0.34910141391446814], 'prec_30': [0.4835317277346768], 'prec_7': [0.42260921106208116], 'precipitation': [0.7594961501084841]}
result {'LIA': [], 'VH': [], 'VV': [], 'VVVH_ratio': [], 'angle': [], 'layover': [], 'no_data_mask': [], 'shadow': []}
result {'LIA': [37.3548234721438], 'VH': [-13.827611086796432], 'VV': [-6.909694212645045], 'VVVH_ratio': [0.16229580459054394], 'angle': [38.27420425415039], 'layover': [1], 'no_data_mask': [1], 'shadow': [1]}
result {'prec_3': [0.012687262561586168], 'prec_30': [0.37328205720121155], 'prec_7': [0.3615907818648817], '

In [17]:
future_result_path.resolve()

PosixPath('/home/dguerrero/1_modules/data/8_training_data/BRG_150710_02_2019-12-02.p')